In [1]:
import uuid
import random

from tqdm import tqdm
from threading import Thread

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *

In [2]:
CLUSTER_IDS = ['172.19.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS)
session = cluster.connect()
session.set_keyspace(KEYSPACE)

cluster.default_retry_policy = RetryPolicy()
cluster.default_reconnection_policy = ExponentialReconnectionPolicy(base_delay=1, max_delay=60, max_attempts=60)

In [3]:
res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    reservation_id = r.reservation_id
    
reservation_id

a4df509d-0274-4f4e-99d7-827befaf8f2a a38279f1-a3b9-4476-92d4-6d9ffc2d032b User 101 Book 2001


UUID('a4df509d-0274-4f4e-99d7-827befaf8f2a')

In [4]:
res = list(get_all_books(session))
book_id, book_name = None, None
print(len(res))
for r in res:
    print(r.book_id, r.book_name, r.is_reserved)
    # if r.is_reserved:
    #     print(r.book_id, r.book_name, r.is_reserved)
    if r.book_name == "Book 1":
        book_id = r.book_id
        book_name = "Book 1"

book_id, book_name

2001
b5be4e26-ff18-4952-aa62-1e8cfde62f77 Book 641 False
3b798d4d-4ae2-46af-b202-4338460b13fb Book 1972 False
4518e38f-8338-40dc-9944-0fd5a89e2e91 Book 1428 False
9120a93b-b6e5-45e4-97bf-c80b4485f475 Book 524 False
49a27c35-e68d-4ec8-bb7b-d05fb5762e21 Book 1522 False
b724141f-e00b-4da6-ac8f-8d7af5da6dd3 Book 1278 False
4b964fa9-7588-43c9-9bec-95af9e3b134e Book 132 False
59b9deac-ae2f-4aa1-8804-f20c69f88f2f Book 51 False
0ac80d42-82ed-49e7-bcdb-34abb09dfa63 Book 1479 False
77cbb4ec-8307-4569-9d36-57a2277bede5 Book 1253 False
b5fbf085-cd73-4de8-81f4-aade0bbda013 Book 1439 False
e9f25c76-20d0-4fb4-9de0-da4fc100fa22 Book 1277 False
bdcf6973-20f0-4ba6-9f49-d29e71729f04 Book 1273 False
c87d2e2a-04a5-4263-a87c-5373c0f0fa46 Book 1687 False
1237a7cb-6706-42d8-910f-510b16a10375 Book 104 False
287935aa-82d5-4c9a-946f-b103135804a7 Book 1313 False
dec3c433-d1b6-40d4-980d-95d72f81aa6e Book 886 False
b53604bc-463f-4cbb-a087-ec3764c233a5 Book 1450 False
6f2248ea-69bd-435c-b38f-5a5478157651 Book 1208 F

(UUID('1b2dd493-5a2f-4208-8955-a76c1184c392'), 'Book 1')

In [5]:
res = list(get_all_users(session))
for r in res:
    if r.user_name == "User 1":
        user_name = "User 1"
        user_id = r.user_id
    # if r.reservation_ids_list is not None:
    #     print(r.user_id, r.user_name, r.reservation_ids_list)
    print(r.user_id, r.user_name, r.reservation_ids_list)

user_name, user_id

e09e27e2-b46c-4dc1-b0ab-b1300b3f8ab1 User 51 None
07188d69-24d8-4433-b7a3-0e0b415c8f4b User 34 None
e972721e-efbd-4c73-aa11-84bc9b1efeae User 63 None
17b97f10-b7b9-42a3-965b-3aa6455c262c User 36 None
e148cd59-6416-4edd-917e-07e6b963b004 User 5 None
1623af1d-fbfa-4885-80ba-a3bcd5795246 User 99 None
57378f4b-9306-4d39-8b4b-32cbb7263d89 User 11 None
da98b667-7b16-42e7-ba4b-e0731a7f377d User 90 None
824fd482-7c20-4a4f-830f-9e493356295f User 96 None
dd9890f9-e8a3-46c4-9ded-bed43f1c4c97 User 93 None
e3430a05-412c-4b62-91d7-c33f83a665a2 User 50 None
142c338f-5eff-4030-abea-27db94c8c138 User 57 None
637c7bf1-5885-4300-a176-1d4dfb97e109 User 12 None
68e99508-7c7c-4fd9-bd53-bd00d9d2fdb0 User 70 None
a38279f1-a3b9-4476-92d4-6d9ffc2d032b User 101 [UUID('a4df509d-0274-4f4e-99d7-827befaf8f2a')]
3b9873a4-65fb-4e49-b8ee-1614b28e37fe User 35 None
68ed6ecf-81b9-4652-8c78-25baab90259f User 79 None
a1a144ae-1169-4a97-80df-c16ac3884469 User 95 None
fdb6ccc7-3a99-49f4-a010-f806ac54dcdd User 78 None
a5c23ce1

('User 1', UUID('2459ca1a-45f5-4372-8745-cadfa5e16f31'))

In [6]:
def test_reserve_cancel_debug(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    # for _ in range(repeat_num):
    for _ in range(3):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

        print("Created")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)

        cancel_reservation(session, reservation_id)
        print("Cancelled")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)


def test_update_reservations_debug(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        assert reservation.book_id != new_book.book_id
        assert reservation.book_name != new_book.book_name

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)
        
        updated_reservation = get_reservation_by_id(session, reservation.reservation_id)
        
        assert updated_reservation.book_id == new_book.book_id
        assert updated_reservation.book_name == new_book.book_name

def test_make_random_requests_two_users_debug(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")
                    
                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   Book:", book)
                print("============================")
                    
                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   User:", user)
                print("============================")

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")
                
                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("============================")

                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Available books:", len(available_books))

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)
                    print("   Selected book:", selected_random_book)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

                print("============================")

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [13]:
def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation_book(session, reservation_id, new_book_id)


def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)


def test_update_reservations(session, num_reservations=1000):
    # Updates a 1000 reservations

    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)

# TODO: currently the same book can be reserved by two users.
# Add random wait time before checking availability/locks/idk
def test_make_all_reservations_two_users(user1, user2):
    # Simulates two users reserving all possible books

    def make_all_reservations(user_id, user_name):
        # Simulates user making all possible reservations one by one

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        # Update available book state
        books = get_all_books(user_session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            
            try:
                add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
            except:
                pass
            
            books = get_all_books(user_session)
            available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()


def test_make_random_requests_two_users(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
            
                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [14]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user.user_name == "User 1":
        user1 = user

    if user.user_name == "User 2":
        user2 = user

user1, user2

(Row(user_id=UUID('2459ca1a-45f5-4372-8745-cadfa5e16f31'), reservation_ids_list=None, user_name='User 1'),
 Row(user_id=UUID('4b55804b-0e88-4a9f-b39c-f3c51e6eb153'), reservation_ids_list=[UUID('f6fa4539-f932-4d5b-8eef-c95d29f6ea55')], user_name='User 2'))

In [15]:
test_make_random_requests_two_users(user1, user2, 10_000)

Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating book reservation status: Error from server: code=2200 [Invalid quer

Exception in thread Thread-14 (make_random_requests):
Traceback (most recent call last):
  File "/home/bartek/miniconda3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/bartek/miniconda3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/home/bartek/miniconda3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_144945/1248246696.py", line 112, in make_random_requests
  File "/home/bartek/big_data/PagePython/PagePython/utils/query_utils.py", line 201, in update_reservation_user
    delete_user_reservation(session, old_user_id, reservation_id, timeout = timeout)
  File "/home/bartek/big_data/PagePython/PagePython/utils/query_utils.py", line 232, in delete_user_reservation
    session.execute(user_reservation_delete_query, timeout=timeout)
  File "cassandra/cluster.py", line 2677, in cassandra.cluster.Session.execute

In [89]:
test_make_all_reservations_two_users(user1, user2)

In [90]:
all_reservations = get_all_reservations(session)

user_1_reservations = 0
user_2_reservations = 0
for reservation in all_reservations:
    if reservation.user_id == user1.user_id:
        user_1_reservations += 1

    if reservation.user_id == user2.user_id:
        user_2_reservations += 1

print(f"User 1 reservation count: {user_1_reservations}")
print(f"User 2 reservation count: {user_2_reservations}")

User 1 reservation count: 1002
User 2 reservation count: 1002


In [91]:
res = list(get_all_reservations(session))
user_1_reservations = [r for r in res if r.user_id == user1.user_id]
user_2_reservations = [r for r in res if r.user_id == user2.user_id]

user_1 = get_user(session, user1.user_id)
user_2 = get_user(session, user2.user_id)

for r in user_1_reservations:
    if r in user_2_reservations:
        print("Error user1 reservation in user2 reservations list")

    if r.reservation_id not in user_1.reservation_ids_list:
        print("Error user1 reservation not in list")

for r in user_2_reservations:
    if r in user_1_reservations:
        print("Error user2 reservation in user1 reservations list")

    if r.reservation_id not in user_2.reservation_ids_list:
        print("Error user2 reservation not in list")

In [92]:
book_ids_1 = set([r.book_id for r in user_1_reservations])
book_ids_2 = set([r.book_id for r in user_2_reservations])

assert len(book_ids_1 & book_ids_2) == 0, "No book should be reserved by both clients at the same time"

AssertionError: No book should be reserved by both clients at the same time

In [93]:
book_ids_1 & book_ids_2

{UUID('35f42768-c776-44b8-8792-bee48d439181'),
 UUID('90c86772-d5b2-4638-82c5-7203c2c58674'),
 UUID('a5158b5f-3b9d-43a2-b059-49aea66b7886'),
 UUID('baa8dda6-ce01-47be-a147-8b3cc55bc330')}

In [96]:
book = list(book_ids_1 & book_ids_2)[0]
for res in get_all_reservations(session):
    if res.book_id == book:
        print(res.reservation_id, res.user_name, res.book_name, res.book_id)

eda0d700-677c-4df0-b291-6d3a82598039 User 1 Book 310 35f42768-c776-44b8-8792-bee48d439181
4fe17768-3b14-4e7b-8a66-142f6eb91b06 User 2 Book 310 35f42768-c776-44b8-8792-bee48d439181


In [94]:
books = get_all_books(session)
available_books = [book for book in books if not book.is_reserved]
available_books

[]

In [32]:
test_update_reservations(session)

100%|██████████| 1000/1000 [00:12<00:00, 77.19it/s]


In [15]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    

100%|██████████| 10000/10000 [03:33<00:00, 46.90it/s]


44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1001


In [17]:
test_same_request(session, reservation_id, book_id)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)

100%|██████████| 10000/10000 [00:44<00:00, 226.46it/s]

44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1


In [ ]:
session.shutdown()
cluster.shutdown()